In [6]:
# from datetime import datetime
import amendments_helper

In [7]:
# Example usage of the function
df = amendments_helper.scrape_amendments(start_date="01/01/2022",end_date="09/30/2024")
df.to_csv('amendments.csv', index=False)

df.head(5)

,Congress,S.Amdt.No,Amends,Sponsor,Latest Action,url,Latest Action Date,Sponsor Party,Outcome,Action Type
0,118th Congress (2023-2024),S.Amdt.3289,H.R.9468,"Paul, Rand [Sen.-R-KY]","Amendment SA 3289, under the order of 9/19/24,...",congress.gov/amendment/118th-congress/senate-a...,09/19/24,R,NA,RCV
1,118th Congress (2023-2024),S.Amdt.1829,H.R.7888,"Paul, Rand [Sen.-R-KY]","Amendment SA 1829, under the order of 4/19/202...",congress.gov/amendment/118th-congress/senate-a...,04/19/24,R,NA,RCV
2,118th Congress (2023-2024),S.Amdt.1841,H.R.7888,"Durbin, Richard J. [Sen.-D-IL]","Amendment SA 1841, as modified, not agreed to ...",congress.gov/amendment/118th-congress/senate-a...,04/19/24,D,NA,RCV
3,118th Congress (2023-2024),S.Amdt.1834,H.R.7888,"Marshall, Roger [Sen.-R-KS]",Amendment SA 1834 not agreed to in Senate by Y...,congress.gov/amendment/118th-congress/senate-a...,04/19/24,R,NA,RCV
4,118th Congress (2023-2024),S.Amdt.1840,H.R.7888,"Lee, Mike [Sen.-R-UT]",Amendment SA 1840 not agreed to in Senate by Y...,congress.gov/amendment/118th-congress/senate-a...,04/19/24,R,NA,RCV


In [8]:
## ROWS THAT NEED TO BE FIXED / LOOKED AT
df[(df["Outcome"] == "INPUT BY HAND") | (df["Action Type"] == "INPUT BY HAND")]

,Congress,S.Amdt.No,Amends,Sponsor,Latest Action,url,Latest Action Date,Sponsor Party,Outcome,Action Type
71,118th Congress (2023-2024),S.Amdt.3292,S.4698,"Peters, Gary C. [Sen.-D-MI]",Senate vitiated previous adoption of amendemen...,congress.gov/amendment/118th-congress/senate-a...,09/23/24,D,A,INPUT BY HAND
104,118th Congress (2023-2024),S.Amdt.1367,S.3000,"Peters, Gary C. [Sen.-D-MI]",NaN,congress.gov/amendment/118th-congress/senate-a...,NaN,D,INPUT BY HAND,INPUT BY HAND
292,117th Congress (2021-2022),S.Amdt.5193,S.734,"Cornyn, John [Sen.-R-TX]",Senate vitiated previous adoption of amendment...,congress.gov/amendment/117th-congress/senate-a...,08/03/22,R,A,INPUT BY HAND
322,117th Congress (2021-2022),S.Amdt.5027,S.Res.533,"Kaine, Tim [Sen.-D-VA]",Amendment SA 5027 proposed by Senator Murphy f...,congress.gov/amendment/117th-congress/senate-a...,04/07/22,D,A,INPUT BY HAND


In [9]:
import pandas as pd

# Assuming df is already defined and populated

# Ensure "Date of Action" is in datetime format, then extract the year
df["Latest Action Date"] = pd.to_datetime(df["Latest Action Date"], format='%m/%d/%y',errors='coerce')
df["Year"] = df["Latest Action Date"].dt.year.astype('Int64')


# Function to generate summary for a specific year
def generate_yearly_summary(df, year):
    # Filter the DataFrame for the specific year
    df_year = df[df["Year"] == year]

    # Count total amendments, R, D, and action types
    l = [
        len(df_year),  # Total Amendments
        len(df_year[df_year["Sponsor Party"] == "R"]),  # R Amendments
        len(df_year[df_year["Sponsor Party"] == "D"]),  # D Amendments
        len(df_year[df_year["Action Type"] == "RCV"]),  # Roll Call Vote (RCV)
        len(df_year[df_year["Action Type"] == "VV"]),  # Voice Vote (VV)
        len(df_year[df_year["Action Type"] == "UC"])   # Unanimous Consent (UC)
    ]
    l_labels = ["Total Amendments", "R Amdts", "D Amdts", "RCV", "VV", "UC"]

    # Breakdown of amendment types
    bills_amended = len(df_year[(df_year["Amends"].str.lower().str.contains("h.r.", na=False)) | 
                                (df_year["Amends"].str.lower().str.contains("treaty", na=False)) | 
                                ((df_year["Amends"].str.lower().str.contains("s.", na=False)) & 
                                 (~df_year["Amends"].str.lower().str.contains("s.j", na=False)) & 
                                 (~df_year["Amends"].str.lower().str.contains("s.res", na=False)))])

    resolutions_amended = len(df_year[df_year["Amends"].str.lower().str.contains("res", na=False)])
    amendments_amended = len(df_year[df_year["Amends"].str.lower().str.contains("amdt", na=False)])
    budget_resolutions = len(df_year[df_year["Amends"].str.lower().str.contains("con", na=False)])

    ltype = [bills_amended, resolutions_amended, amendments_amended, budget_resolutions]
    ltype_labels = ["Bills Amended", "Resolutions Amended", "Amdts Amended", "Budget Resolutions"]

    # Create the summary DataFrame for the year
    summary = pd.DataFrame(ltype, index=ltype_labels, columns=[f"{year}"])

    # Add the first set of summary data for total amendments and action types
    summary = pd.concat([summary, pd.DataFrame(l, index=l_labels, columns=[f"{year}"])])

    # Return the summary DataFrame
    return summary

# Get a list of unique years in the data
years = df["Year"].dropna().unique()

# Generate summaries for all years and concatenate them
summary_by_year = pd.concat([generate_yearly_summary(df, year) for year in years], axis=1)
summary_by_year = summary_by_year.sort_index(axis=1)

# Display the final summary broken down by year
summary_by_year


,2022,2023,2024
Bills Amended,143,125,46
Resolutions Amended,20,11,3
Amdts Amended,38,72,1
Budget Resolutions,0,2,0
Total Amendments,165,134,49
R Amdts,83,81,24
D Amdts,78,51,25
RCV,54,62,16
VV,5,40,3
UC,104,32,29


In [10]:
## OLD CODE

# results = pd.DataFrame(test, columns = ["S.Amdt.No","Amends","Sponsor","Sponsor Party","Date of Action","Outcome","Action Type"])
# df = results
# l=[len(df),len(df[df["Sponsor Party"]=="R"]),len(df[df["Sponsor Party"]=="D"]),len(df[df["Action Type"]=="RCV"]),len(df[df["Action Type"]=="VV"]),len(df[df["Action Type"]=="UC"])]
# l_labels = ["Total Amendments","R Amdts","D Amdts","RCV","VV","UC"]
# ltype = [len(df[df["Amends"].str.contains("H.R.")]) + len(df[df["Amends"].str.contains("Treaty")]) + len(df[~df["Amends"].str.contains("S.J")][~df["Amends"].str.contains("S.Res")][df["Amends"].str.contains("S.")]),
#             len(df[df["Amends"].str.contains("Res")]),
#             len(df[df["Amends"].str.contains("Amdt")]),
#             len(df[df["Amends"].str.contains("con")])]
# ltype_labels = ["Bills Amended","Resolutions Amended","Amdts Amended","Budget Resolutions"]

# summary = pd.DataFrame(ltype,index=ltype_labels,columns=["Summary Data for Senate Amendments"])
# summary = pd.concat([summary,pd.DataFrame(l,index=l_labels,columns=["Summary Data for Senate Amendments"])])
# summary = pd.concat([summary,pd.DataFrame([(datestart.strftime("%m/%d/%Y")+" through "+dateend.strftime("%m/%d/%Y"))],index=["Date Range"],columns=["Summary Data for Senate Amendments"])])  

# summary
# return(results,summary)